# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

F. Walter  ->  F. Walter  |  ['F. Walter']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
J. Li  ->  J. Li  |  ['J. Li']


K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
M. Schirmer  ->  M. Schirmer  |  ['M. Schirmer']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
Arxiv has 75 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2503.15594


extracting tarball to tmp_2503.15594...

 done.


/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2503.15594/midisngdeep_zgt14.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'appendixA' from 'tmp_2503.15594/appendixA.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'fig8' from 'tmp_2503.15594/fig8.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_

...

extracting tarball to tmp_2503.16204...

 done.
Retrieving document from  https://arxiv.org/e-print/2503.16367


extracting tarball to tmp_2503.16367...

 done.


Found 96 bibliographic references in tmp_2503.16367/main.bbl.
Issues with the citations
syntax error in line 88: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.15594-b31b1b.svg)](https://arxiv.org/abs/2503.15594) | **The rise of the galactic empire: luminosity functions at $z\sim17$ and $z\sim25$ estimated with the MIDIS$+$NGDEEP ultra-deep JWST/NIRCam dataset**  |
|| P. G. Pérez-González, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2025-03-21*|
|*Comments*| *Submitted to ApJ. Comments welcome*|
|**Abstract**|            We present a sample of six F200W and three F277W dropout sources identified as $16<z<25$ galaxy candidates based on the deepest JWST/NIRCam data to date, provided by the MIRI Deep Imaging Survey (MIDIS) and the Next Generation Deep Extragalactic Exploratory Public survey (NGDEEP), reaching 5$\sigma$ depths of $\sim31.5$ mag (AB) at $\geq2$ $\mu$m. We estimate ultraviolet (UV) luminosity functions and densities at $z\sim17$ and $z\sim25$. We find that the number density of galaxies with absolute magnitudes $-19<M_\mathrm{UV}<-18$ (AB) at $z\sim17$ ($z\sim25$) is a factor of 4 (25) smaller than at $z\sim12$; a similar evolution is observed for the luminosity density. Compared to state-of-the-art galaxy simulations, we find the need for an enhanced UV-photon production at $z=17-25$ in $\mathrm{M}_\mathrm{DM}=10^{8.5-9.5}$ M$_\odot$ dark matter halos, maybe provided by an increase in the star formation efficiency at early times and/or by intense bursts fed by very low metallicity or primordial gas. There are few robust theoretical predictions for the evolution of galaxies above $z\sim20$ in the literature, however, the continuing rapid drop in the halo mass function suggests more rapid evolution than we observe if photon production efficiencies remained constant. Our $z>16$ galaxy candidates present mass-weighted ages around 30 Myr, and attenuations $\mathrm{A(V)}<0.1$ mag. Their average stellar mass is $\mathrm{M}_\bigstar\sim10^{7}\,\mathrm{M}_\odot$, implying a star formation efficiency (stellar-to-baryon mass fraction) around 10%. We find three galaxies with very blue UV spectral slopes ($\beta\sim-3$) compatible with low metallicity or Pop III and young ($\lesssim10$ Myr) stars and/or high escape fractions of ionizing photons, the rest presenting slopes $\beta\sim-2.5$ similar to $z=10-12$ samples.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.15635-b31b1b.svg)](https://arxiv.org/abs/2503.15635) | **Euclid preparation. Spatially resolved stellar populations of local galaxies with Euclid: a proof of concept using synthetic images with the TNG50 simulation**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-03-21*|
|*Comments*| *31 pages, 18 figures, submitted to A&A*|
|**Abstract**|            The European Space Agency's Euclid mission will observe approximately 14,000 $\rm{deg}^{2}$ of the extragalactic sky and deliver high-quality imaging for many galaxies. The depth and high spatial resolution of the data will enable a detailed analysis of stellar population properties of local galaxies. In this study, we test our pipeline for spatially resolved SED fitting using synthetic images of Euclid, LSST, and GALEX generated from the TNG50 simulation. We apply our pipeline to 25 local simulated galaxies to recover their resolved stellar population properties. We produce 3 types of data cubes: GALEX + LSST + Euclid, LSST + Euclid, and Euclid-only. We perform the SED fitting tests with two SPS models in a Bayesian framework. Because the age, metallicity, and dust attenuation estimates are biased when applying only classical formulations of flat priors, we examine the effects of additional priors in the forms of mass-age-$Z$ relations, constructed using a combination of empirical and simulated data. Stellar-mass surface densities can be recovered well using any of the 3 data cubes, regardless of the SPS model and prior variations. The new priors then significantly improve the measurements of mass-weighted age and $Z$ compared to results obtained without priors, but they may play an excessive role compared to the data in determining the outcome when no UV data is available. The spatially resolved SED fitting method is powerful for mapping the stellar populations of galaxies with the current abundance of high-quality imaging data. Our study re-emphasizes the gain added by including multiwavelength data from ancillary surveys and the roles of priors in Bayesian SED fitting. With the Euclid data alone, we will be able to generate complete and deep stellar mass maps of galaxies in the local Universe, thus exploiting the telescope's wide field, NIR sensitivity, and high spatial resolution.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.16367-b31b1b.svg)](https://arxiv.org/abs/2503.16367) | **Euclid: Early Release Observations -- Interplay between dwarf galaxies and their globular clusters in the Perseus galaxy cluster**  |
|| T. Saifollahi, et al. -- incl., <mark>M. Schirmer</mark>, <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-03-21*|
|*Comments*| *23 pages, 17 figures, Submitted to A&A*|
|**Abstract**|            We present an analysis of globular clusters (GCs) of dwarf galaxies in the Perseus galaxy cluster to explore the relationship between dwarf galaxy properties and their GCs. Our focus is on GC numbers ($N_{\rm GC}$) and GC half-number radii ($R_{\rm GC}$) around dwarf galaxies, and their relations with host galaxy stellar masses ($M_*$), central surface brightnesses ($\mu_0$), and effective radii ($R_{\rm e}$). Interestingly, we find that at a given stellar mass, $R_{\rm GC}$ is almost independent of the host galaxy $\mu_0$ and $R_{\rm e}$, while $R_{\rm GC}/R_{\rm e}$ depends on $\mu_0$ and $R_{\rm e}$; lower surface brightness and diffuse dwarf galaxies show $R_{\rm GC}/R_{\rm e}\approx 1$ while higher surface brightness and compact dwarf galaxies show $R_{\rm GC}/R_{\rm e}\approx 1.5$-$2$. This means that for dwarf galaxies of similar stellar mass, the GCs have a similar median extent; however, their distribution is different from the field stars of their host. Additionally, low surface brightness and diffuse dwarf galaxies on average have a higher $N_{\rm GC}$ than high surface brightness and compact dwarf galaxies at any given stellar mass. We also find that UDGs (ultra-diffuse galaxies) and non-UDGs have similar $R_{\rm GC}$, while UDGs have smaller $R_{\rm GC}/R_{\rm e}$ (typically less than 1) and 3-4 times higher $N_{\rm GC}$ than non-UDGs. Examining nucleated and not-nucleated dwarf galaxies, we find that for $M_*>10^8M_{\odot}$, nucleated dwarf galaxies seem to have smaller $R_{\rm GC}$ and $R_{\rm GC}/R_{\rm e}$, with no significant differences between their $N_{\rm GC}$, except at $M_*<10^8M_{\odot}$ where the nucleated dwarf galaxies tend to have a higher $N_{\rm GC}$. Lastly, we explore the stellar-to-halo mass ratio (SHMR) of dwarf galaxies and conclude that the Perseus cluster dwarf galaxies follow the expected SHMR at $z=0$ extrapolated down to $M_*=10^6M_{\odot}$.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.16145-b31b1b.svg)](https://arxiv.org/abs/2503.16145) | **Distributions and Physical Properties of Molecular Clouds in the G24 Region of the Milky Way**  |
|| T. Yang, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-03-21*|
|*Comments*| **|
|**Abstract**|            We report the spatial distribution and physical characteristics of molecular clouds in the G24 region, which is located near the intersection of the Milky Way's Galactic bar with the Norma arm and the 3 kpc arm. Utilizing molecular line data from the Milky Way Imaging Scroll Painting (MWISP) project, including $^{12}$CO, $^{13}$CO, and C$^{18}$O, along with our own observations of HCO$^{+}$ line using the Purple Mountain Observatory (PMO) 13.7 m telescope, we have revealed the complex architecture of molecular clouds in the G24 region. Seven giant molecular clouds, each with a mass exceeding $10^4$ $M_\odot$ and a typical H$_2$ column density of $10^{21}$ cm$^{-2}$, have been identified through observations of CO and its isotopes. The conversion factor $X_{\text{CO}}$ for the G24 region is estimated to be 8.25 $\times$ 10$^{19}$ cm$^{-2}$ (K km s$^{-1}$)$^{-1}$, aligning with the typical values observed in other regions. Adopting the GaussClumps algorithm, we have identified a total of 257, 201, and 110 clumps in $^{12}$CO, $^{13}$CO and C$^{18}$O within G24 region, respectively. The derived physical properties (including effective radius, mass, and virial parameter) indicate that the majority of these clumps are gravitationally bound, with a subset possessing the potential to form massive stars. Examination of gas infall activities within these clumps further suggests ongoing massive star formation. The complex physical and kinematic environment, shaped by the G24 region's unique location within the Milky Way, has limited the clear detection of gas outflows.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.16204-b31b1b.svg)](https://arxiv.org/abs/2503.16204) | **Resolution of a paradox: SDSS J1257+5428 can be explained as a descendant of a cataclysmic variable with an evolved donor**  |
|| D. Belloni, M. R. Schreiber, <mark>K. El-Badry</mark> |
|*Appeared on*| *2025-03-21*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**|            The existence of the binary system SDSS J1257+5428 has been described as paradoxical. Here we investigate under which conditions SDSS J1257+5428 could be understood as a descendant of a cataclysmic variable with an evolved donor star, which is a scenario that has never been explored in detail. We used the BSE code for pre-common-envelope (CE) evolution and the MESA code for post-CE evolution to run binary evolution simulations and searched for potential formation pathways for SDSS J1257+5428 that lead to its observed characteristics. For the post-CE evolution, we adopted a boosted version of the CARB model. We find that SDSS J1257+5428 can be explained as a post-cataclysmic-variable system if (i) the progenitor of the extremely low-mass WD was initially a solar-type star that evolved into a subgiant before the onset of mass transfer and underwent hydrogen shell flashes after the mass transfer stopped, (ii) the massive WD was highly or entirely rejuvenated during the cataclysmic variable evolution, and (iii) magnetic braking was strong enough to make the evolution convergent. In this case, the torques due to magnetic braking need to be stronger than those provided by the CARB model by a factor of ${\sim100}$. We conclude that SDSS J1257+5428 can be reasonably well explained as having originated from a cataclysmic variable that hosted an evolved donor star and should no longer be regarded as paradoxical. If our formation channel is correct, our findings provide further support that stronger magnetic braking acts on progenitors of (i) close detached WD binaries, (ii) close detached millisecond pulsar with extremely low-mass WDs, (iii) AM CVn binaries, and (iv) ultra-compact X-ray binaries, in comparison to the magnetic braking strength required to explain binaries hosting main-sequence stars and single main-sequence stars.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2503.15594/./Figures/masstolight_v20250307_v3.13.png', 'tmp_2503.15594/./Figures/surveys_v3.12.png', 'tmp_2503.15594/./Figures/midisred0019207.png']
copying  tmp_2503.15594/./Figures/masstolight_v20250307_v3.13.png to _build/html/
copying  tmp_2503.15594/./Figures/surveys_v3.12.png to _build/html/
copying  tmp_2503.15594/./Figures/midisred0019207.png to _build/html/
exported in  _build/html/2503.15594.md
    + _build/html/tmp_2503.15594/./Figures/masstolight_v20250307_v3.13.png
    + _build/html/tmp_2503.15594/./Figures/surveys_v3.12.png
    + _build/html/tmp_2503.15594/./Figures/midisred0019207.png
found figures ['tmp_2503.15635/./comp_fitsim_GLE_fsps_new2.png', 'tmp_2503.15635/./comp_fitsim_GLE_bc16_new2.png', 'tmp_2503.15635/./comp_fitsim_LE_fsps_new2.png', 'tmp_2503.15635/./comp_fitsim_LE_bc16_new2.png', 'tmp_2503.15635/./comp_fitsim_E_fsps_new2.png', 'tmp_2503.15635/./comp_fitsim_E_bc16_new2.png']
copying  tmp_2503.15635/./comp_fitsim_GLE_fsps_new2.png to _buil

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# The rise of the galactic empire: luminosity functions  at $\boldmath{$z\sim17$}$ and $\boldmath{$z\sim25$}$ estimated with the MIDIS$+$NGDEEP ultra-deep JWST/NIRCam dataset

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.15594-b31b1b.svg)](https://arxiv.org/abs/2503.15594)<mark>Appeared on: 2025-03-21</mark> -  _Submitted to ApJ. Comments welcome_

</div>
<div id="authors">

P. G. Pérez-González, et al. -- incl., <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** We present a sample of six F200W and three F277W dropout sources identified as $16<z<25$ galaxy candidates based on the deepest JWST/NIRCam data to date, provided by the MIRI Deep Imaging Survey (MIDIS) and the Next Generation Deep Extragalactic Exploratory Public survey  (NGDEEP), reaching 5 $\sigma$ depths of $\sim31.5$ mag (AB) at $\geq2$  $\mu$ m. We estimate ultraviolet (UV) luminosity functions and densities at $z\sim17$ and $z\sim25$ . We find that the number density of galaxies with absolute magnitudes $-19<M_\mathrm{UV}<-18$ (AB) at $z\sim17$ ( $z\sim25$ )is a factor of 4 (25) smaller than at $z\sim12$ ; a similar evolution is observed for the luminosity density. Compared to state-of-the-art galaxy simulations, we find the need for an enhanced UV-photon production at $z=17-25$ in $\mathrm{M}_\mathrm{DM}=10^{8.5-9.5}$ M $_\odot$ dark matter halos, maybe provided by an increase in the star formation efficiency at early times and/or by intense bursts fed by very low metallicity or primordial gas.There are few robust theoretical predictions for the evolution of galaxies above $z\sim 20$ in the literature, however, the continuing rapid drop in the halo mass function suggests more rapid evolution than we observe if photon production efficiencies remained constant.Our $z>16$ galaxy candidates present mass-weighted ages around 30 Myr, and attenuations $\mathrm{A(V)}<0.1$ mag.  Their average stellar mass is $\mathrm{M}_\bigstar\sim10^{7} \mathrm{M}_\odot$ , implying a star formation efficiency (stellar-to-baryon mass fraction) around 10 \% .  We find three galaxies with very blue UV spectral slopes ( $\beta\sim-3$ ) compatible with low metallicity or Pop III and young ( $\lesssim10$ Myr) stars and/or high escape fractions of ionizing photons, the rest presenting slopes $\beta\sim-2.5$ similar to $z=10-12$ samples.

</div>

<div id="div_fig1">

<img src="tmp_2503.15594/./Figures/masstolight_v20250307_v3.13.png" alt="Fig9" width="100%"/>

**Figure 9. -** Stellar mass-to-light ratios as a function of halo mass for redshifts $z\sim9$ to $z\sim25$. The left y-axis give UV luminosity densities in solar units (assuming an AB absolute magnitude for the Sun $M^{UV}_\odot=17.30$,  ([ and Willmer 2018]()) ), the right y-axis in CGS units. The vertical axes could also be read in terms of dark matter mass-to-light ratios considering the baryon-to-dark matter density ratio ($\sim19$\%) and the star formation efficiency, defined as stellar mass divided by baryon mass, assumed to be 15\% in the plot. The curved lines provide results based on an abundance matching comparison between luminosity functions from this work and [Pérez-González, Costantin and Langeroodi (2023)]() and dark matter mass functions from [Yung, Somerville and Nguyen (2024)](). Thick segments show the calculations based on the range of absolute magnitudes actually probed by those works, while the thin lines show results based on extrapolated Schechter fits. We differentiate between the faint/light ends of the luminosity and mass functions, which might be well characterized by the current data with a faint-end slope (where we use a continuous line) and the bright/massive ends, whose behavior is more complex and unconstrained for most redshifts above $z=10$(where we used a dotted line). Oblique lines show the UV absolute magnitudes corresponding to the dark matter masses according to the abundance matching exercise, running from -17 to -20 every one magnitude (the extreme values are marked).  Horizontal lines show mass-to-light ratios for a compilation of relevant stellar populations models from [Zackrisson, et. al (2011)](), with metallicities spanning expected values from 1/5 Z$_\odot$ to Pop-III stars, and different IMFs including [ and Kroupa (2001)]() and top-heavy parametrizations. Lines in black and light gray show predictions for 30 and 1 Myr old instantaneous bursts. We note that the models with the most extreme top-heavy IMF do not extend to 30 Myr, those massive stars die earlier. This effect also affects the relative mass-to-light ratios between the Kroupa and top-heavy IMFs for 30 Myr (reversed compared to 1 Myr predictions). (*fig:m/l*)

</div>
<div id="div_fig2">

<img src="tmp_2503.15594/./Figures/surveys_v3.12.png" alt="Fig2" width="100%"/>

**Figure 2. -** Compilation of spectroscopically confirmed galaxies (red hexagons,  ([Dickinson and Finkelstein 2023](), [Curtis-Lake, Carniani and Cameron 2023](), [Castellano, Napolitano and Fontana 2024](), [Carniani and Hainline 2024]()) ) and photometric samples (including  ([Leung, Bagley and Finkelstein 2023]())  -green pointing-up triangles-,  ([Finkelstein, Leung and Bagley 2024]())  -pointing-down triangle-,  ([Pérez-González, Costantin and Langeroodi 2023]())  -green stars-,  ([Kokorev, Atek and Chisholm 2024]())  -red squares-, and  ([Casey, Akins and Shuntov 2024]())  -green crosses-) of galaxies at $z>11$. The $16<z<25$ galaxy candidates presented in this work are plotted with green and orange diamonds, the color representing the band used in the plot, the one closer to and redward of the Lyman break. We also show their magnitude and photometric redshift uncertainties. Lines show the expected magnitudes as a function of redshift of a very young (1 Myr old) starburst with stellar mass $M_\star=10^8 M_\odot$ according to the [Bruzual and Charlot (2003)]() models (thick dotted line) and a less massive $M_\star=10^7 M_\odot$ starburst according to the [Eldridge, Stanway and Xiao (2017)]() BPASS models (thin dotted line). The expected magnitudes for a more extended star formation history (50 Myr constant star formation population) and a dormant galaxy (which experienced an instantaneous burst 30 Myr before the observation) are also shown (thick continuous and dashed-dotted lines, respectively). All models assume no dust. Shaded regions show the 5$\sigma$ depths of the major galaxy surveys used in the identification of $z>11$ galaxy candidates. The darkest regions refer to the dataset used in this paper (MIDIS24, which stands for the 2024 depth, including NGDEEP). The color for all data points, lines, and shaded regions indicates the NIRCam band considered.  (*fig:depths*)

</div>
<div id="div_fig3">

<img src="tmp_2503.15594/./Figures/midisred0019207.png" alt="Fig10" width="100%"/>

**Figure 10. -** One of the galaxies selected in the $z\sim17$ sample. The figure shows the same information depicted for an example galaxy in the main text (Figure \ref{fig:sed_examples}). (*fig:z17_2*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.15594"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\pixedfit}{\texttt{piXedfit}}$
$\newcommand{\bagpipes}{\textsc{BAGPIPES}}$
$\newcommand{\sextractor}{\textsc{SExtractor}}$
$\newcommand{\sep}{\textsc{sep}}$
$\newcommand{\galfit}{\textsc{GALFIT}}$
$\newcommand{\skirt}{\textsc{SKIRT}}$
$\newcommand{\photutils}{\textsc{photutils}}$
$\newcommand{\scipy}{\textsc{scipy}}$
$\newcommand{\magphys}{\textsc{MAGPHYS}}$
$\newcommand{\galex}{GALEX}$
$\newcommand{\lsst}{LSST}$
$\newcommand{\euclid}{\textit{Euclid}}$
$\newcommand{\wise}{WISE}$
$\newcommand{\jwst}{JWST}$
$\newcommand{\hst}{HST}$
$\newcommand{\hsc}{HSC}$
$\newcommand{\des}{DES}$
$\newcommand{\ie}{I_{\rm E}}$
$\newcommand{\ye}{Y_{\rm E}}$
$\newcommand{\je}{J_{\rm E}}$
$\newcommand{\he}{H_{\rm E}}$
$\newcommand{\mass}{M_{*}}$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\massd}{\Sigma_{*}}$
$\newcommand{\sfrd}{\Sigma_{\rm SFR}}$
$\newcommand{\gledc}{G + L + E}$
$\newcommand{\ledc}{L + E}$
$\newcommand{\logten}{\ensuremath{\log_{10}}}$
$\newcommand{\orcid}[1]$
$\newcommand{\msun}{\mbox{M_{\rm \odot}}}$
$\newcommand{\om}{\mbox{\Omega_{\rm m}}}$
$\newcommand{\ol}{\mbox{\Omega_{\rm \Lambda}}}$
$\newcommand{\hnot}{\mbox{H_{\rm 0}}}$</div>



<div id="title">

# $\euclid$ preparation: Spatially resolved stellar populations of local galaxies with $\euclid$: a proof of concept using synthetic images with the TNG50 simulation

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.15635-b31b1b.svg)](https://arxiv.org/abs/2503.15635)<mark>Appeared on: 2025-03-21</mark> -  _31 pages, 18 figures, submitted to A&A_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** The European Space Agency's $\euclid$ mission will observe approximately $14 000  \rm{deg}^{2}$ of the extragalactic sky and deliver high-quality imaging for a large number of galaxies. The depth and high spatial resolution of the data will enable a detailed analysis of stellar population properties of local galaxies through spatially resolved spectral energy distribution (SED) fitting.In this study, we test our pipeline for spatially resolved SED fitting using synthetic images of $\euclid$ , $\lsst$ , and $\galex$ generated from the TNG50 simulation using the SKIRT 3D radiative transfer code. Our pipeline uses functionalities in $\pixedfit$ for processing the simulated data cubes and carrying out SED fitting. We apply our pipeline to 25 simulated galaxies in the local universe to recover their resolved stellar population properties. For each galaxy, we produce 3 types of data cubes: $\galex$ + $\lsst$ + $\euclid$ , $\lsst$ + $\euclid$ , and $\euclid$ -only.We perform the SED fitting tests with two stellar population synthesis (SPS) models in a Bayesian framework. Because the age, metallicity ( $Z$ ), and dust attenuation estimates are biased when applying only classical formulations of flat priors (even with the combined $\galex$ + $\lsst$ + $\euclid$ data), we examine the effects of additional physically motivated priors in the forms of mass-age and mass-metallicity relations, constructed using a combination of empirical and simulated data.Stellar-mass surface densities can be recovered well using any of the 3 data cubes, regardless of the SPS model and prior variations. The new priors then significantly improve the measurements of mass-weighted age and $Z$ compared to results obtained without priors, but they may play an excessive role compared to the data in determining the outcome when no ultraviolet (UV) data is available.Compared to varying the spectral extent of the data cube or including and discarding the additional priors, replacing one SPS model family with the other has little effect on the results.The spatially resolved SED fitting method is powerful for mapping the stellar population properties of many galaxies with the current abundance of high-quality imaging data. Our study re-emphasizes the gain added by including multiwavelength data from ancillary surveys and the roles of priors in Bayesian SED fitting.With the $\euclid$ data alone, we will be able to generate complete and deep stellar mass maps of galaxies in the local Universe, thus exploiting the telescope's wide field, near-infrared (NIR) sensitivity, and high spatial resolution.

</div>

<div id="div_fig1">

<img src="tmp_2503.15635/./comp_fitsim_GLE_fsps_new2.png" alt="Fig12.1" width="50%"/><img src="tmp_2503.15635/./comp_fitsim_GLE_bc16_new2.png" alt="Fig12.2" width="50%"/>

**Figure 12. -** Comparisons of the spatially resolved stellar population properties derived from SED fitting on the $\gledc$ data cubes (ordinate) and the true properties (abscissa). These include all spatial bins from the whole sample of galaxies. We focus on four main properties in this test analysis: $\mass$d, $Z$, age, and $\sfrd$. The top row (a) shows results obtained with the FSPS model, while the bottom row (b) shows results using the BC16 model. The blue and red colors represent results obtained with and without applying the mass-$Z$-age prior. (*fig:comp_simfits_GLE*)

</div>
<div id="div_fig2">

<img src="tmp_2503.15635/./comp_fitsim_LE_fsps_new2.png" alt="Fig13.1" width="50%"/><img src="tmp_2503.15635/./comp_fitsim_LE_bc16_new2.png" alt="Fig13.2" width="50%"/>

**Figure 13. -** Similar to Fig. \ref{fig:comp_simfits_GLE} but for results of SED fitting on the $\ledc$ data cubes. (*fig:comp_simfits_LE*)

</div>
<div id="div_fig3">

<img src="tmp_2503.15635/./comp_fitsim_E_fsps_new2.png" alt="Fig14.1" width="50%"/><img src="tmp_2503.15635/./comp_fitsim_E_bc16_new2.png" alt="Fig14.2" width="50%"/>

**Figure 14. -** Similar to Fig. \ref{fig:comp_simfits_GLE} but for results of SED fitting on the E data cube. (*fig:comp_simfits_E*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.15635"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]{\orcidlink{#1}}$</div>



<div id="title">

# $\Euclid$: Early Release Observations -- Interplay between dwarf galaxies and their globular clusters in the Perseus galaxy cluster$\thanks{This paper is published on       behalf of the Euclid Consortium}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.16367-b31b1b.svg)](https://arxiv.org/abs/2503.16367)<mark>Appeared on: 2025-03-21</mark> -  _23 pages, 17 figures, Submitted to A&A_

</div>
<div id="authors">

T. Saifollahi, et al. -- incl., <mark>M. Schirmer</mark>, <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** We present an analysis of globular clusters (GCs) of dwarf galaxies in the Perseus galaxy cluster to explore the relationship between dwarf galaxy properties and their GCs, which offer important clues on the origin and subsequent evolution of dwarf galaxies. Our focus is on GC numbers ( $N_{\rm GC}$ ) and GC half-number radii ( $R_{\rm GC}$ ) around dwarf galaxies, and their relations with host galaxy stellar masses ( $M_{\ast}$ ), central surface brightnesses ( $\mu_0$ ), and effective radii ( $R_{\rm e}$ ). Interestingly, we find that at a given stellar mass, $R_{\rm GC}$ is almost independent of the host galaxy $\mu_0$ and $R_{\rm e}$ , while $R_{\rm GC}/R_{\rm e}$ depends on $\mu_0$ and $R_{\rm e}$ ; lower surface brightness and diffuse dwarf galaxies show $R_{\rm GC}/R_{\rm e}\approx 1$ while higher surface brightness and compact dwarf galaxies show $R_{\rm GC}/R_{\rm e} \approx $ $1.5$ -- $2$ . This means that for dwarf galaxies of similar stellar mass, the GCs have a similar median extent; however, their distribution is different from the field stars of their host. Additionally, low surface brightness and diffuse dwarf galaxies on average have a higher $N_{\rm GC}$ than high surface brightness and compact dwarf galaxies at any given stellar mass. We also find that UDGs (ultra-diffuse galaxies) and non-UDGs in the sample have similar $R_{\rm GC}$ , while UDGs have smaller $R_{\rm GC}/R_{\rm e}$ (typically less than 1) and 3--4 times higher $N_{\rm GC}$ than non-UDGs. Furthermore, examining nucleated versus not-nucleated dwarf galaxies, we find that for $M_{\ast} > 10^{8} \si{M_{\odot}}$ , nucleated dwarf galaxies seem to have smaller $R_{\rm GC}$ and $R_{\rm GC}/R_{\rm e}$ , with no significant differences seen between their $N_{\rm GC}$ , except at $M_{\ast} < 10^{8} \si{M_{\odot}}$ where the nucleated dwarf galaxies tend to have a higher $N_{\rm GC}$ . Lastly, we explore the stellar-to-halo mass ratio (SHMR) of dwarf galaxies (halo mass based on $N_{\rm GC}$ ) and conclude that the Perseus cluster dwarf galaxies follow the expected SHMR at $z=0$ extrapolated down to $M_{\ast} = 10^{6} \si{M_{\odot}}$ .

</div>

<div id="div_fig1">

<img src="tmp_2503.16367/./figures/results/ERO-PERSEUS_stacked-radial-profiles-stellarmass_all.png" alt="Fig3" width="100%"/>

**Figure 3. -** GC half-number radius ($R_{\rm GC}$, top panel), and ratio between the GC half-number radius and the average of the host-galaxies effective radius ($R_{\rm GC} / \langle R_{\rm e} \rangle$, bottom panel), as functions of the stellar mass of the host galaxies ($M_{\ast}$) for all dwarf galaxies in the sample. The values are estimated from stacked GC radial profiles that we fit with a Sérsic function, with a Sérsic index fixed to $n=1$(see text). The shaded region around each line represent the 68\% percentiles of the estimated average value. The dashed lines show the trends one would expect from a simple extension of the equations in \citet{lim2024}, that were derived for GCs in massive galaxies, to our lower mass regime (an extrapolation beyond the range examined by these authors).  (*dwarfs-stacked-rgc-all*)

</div>
<div id="div_fig2">

<img src="tmp_2503.16367/./figures/results/ERO-PERSEUS_stacked-radial-profiles+RE.png" alt="Fig4" width="100%"/>

**Figure 4. -** Similar to Fig. \ref{dwarfs-stacked-rgc-all} for different dwarf galaxy categories. The two top panels represent the estimations for UDGs and non-UDGs (in purple and grey), the two middle panels for LSB and HSB dwarf galaxies (in red and blue), and the bottom two panels for DIFF and COMP dwarf galaxies (in orange and green). (*dwarfs-stacked-rgc*)

</div>
<div id="div_fig3">

<img src="tmp_2503.16367/./figures/results/ERO-PERSEUS_dwarf-division.png" alt="Fig2" width="100%"/>

**Figure 2. -** Division of the dwarf galaxy sample into high surface brightness dwarf galaxies (HSB, blue points), low surface brightness dwarf galaxies (LSB, red points), compact dwarf galaxies (COMP, green points), and diffuse dwarf galaxies (DIFF, orange points) based on their central surface brightness ($\mu_0$) and effective radius ($R_{\rm e}$). A distance of 72 Mpc was assumed to convert angular to physical scale. This division is stellarmass dependent and is indicated with the black dashed curve. Coloured points represent the dwarf galaxies that have been used for the analysis in this paper. Those dwarf galaxies that also satisfy the UDG criteria in $\mu_{0, \IE}$ and $R_{\rm e}$(shown with dashed horizontal lines) are indicated with the black circles around the coloured circles. Grey points show those dwarf galaxies that have been removed from the sample after initial assessment (as was mentioned in Sect. \ref{sc:Data-dwarf-cat}). (*dwarfs-division*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.16367"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

396  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

35  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

29  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
